In [3]:
import kwant
import kwant.continuum
import scipy.sparse.linalg
import scipy.linalg
import numpy as np
import time

## 24 hour format ##
def print_t(str_):
  print( "[" + time.strftime("%Y-%m-%d %H:%M:%S") + "] " + str(str_))
  
# For plotting
import matplotlib as mpl
from matplotlib import pyplot as plt

plt.rcParams['text.usetex'] = True
plt.rcParams['font.family'] = 'serif'

s0 = np.eye(2)
sx = np.array([[0,1], [1,0]])
sy = np.array([[0, -1j],[1j, 0]])
sz = np.array([[1,0],[0,-1]])

def qsh_system(a=5, Lx=5000, Ly=500):
  
    hamiltonian = """
    (C-D*(k_x**2+k_y**2))*identity(4)
    + (M-B*(k_x**2+k_y**2))*kron(sigma_0, sigma_z)
    + A*k_x*kron(sigma_z, sigma_x)
    + A*k_y*kron(sigma_z, sigma_y)
    + V(x,y)*identity(4)
    """
  
    template = kwant.continuum.discretize(hamiltonian, grid=a)

    def shape(site):
        (x, y) = site.pos
        return (0 <= y < Ly and 0 <= x < Lx)

    def lead_shape(site):
        (x, y) = site.pos
        return (0 <= y < Ly)

    syst = kwant.Builder()
    syst.fill(template, shape, (0, 0))

    lead = kwant.Builder(kwant.TranslationalSymmetry([-a, 0]))
    lead.fill(template, lead_shape, (0, 0))

    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())

    syst = syst.finalized()
    return syst

/Users/nagydaniel/miniconda3/lib/python3.6/site-packages/kwant/solvers/default.py:18: RuntimeWarning: MUMPS is not available, SciPy built-in solver will be used as a fallback. Performance can be very poor in this case.
  "Performance can be very poor in this case.", RuntimeWarning)


In [ ]:
def no_disorder(x, y):
  return 0

A = 3.645
B = -68.6
C = 0
D = -51.2

M_1 = 1e-3
M_2 = -10e-3

#params_1 = dict(A=A, B=B, C=C, D=D, M=M_1, V=no_disorder)
#params_2 = dict(A=A, B=B, C=C, D=D, M=M_2, V=no_disorder)

params_1 = dict(A = 364.5e-3, B=-686e-3, C=0.0, D=-512e-3, M=1.2e-2, V=no_disorder)
params_2 = dict(A = 364.5e-3, B=-686e-3, C=0.0, D=-512e-3, M=-1e-3, V=no_disorder)

e_1 = -20e-3
e_2 = 25e-3
e_3 = 9e-3

a=5
Lx=5000
Ly=500

In [ ]:
from matplotlib.ticker import AutoMinorLocator
syst = qsh_system(a, Lx, Ly)

fig, ax = plt.subplots(figsize=(8,6), dpi=196)
kwant.plotter.bands(syst.leads[0], params=params_1,
                    momenta=np.linspace(-0.3, 0.3, 150),
                    ax=ax);

ax.axhline(e_1, color='g', linestyle='--', label="{} meV".format(1e3*e_1))
ax.axhline(e_2, color='purple', linestyle='--', label="{} meV".format(1e3*e_2))
ax.axhline(e_3, color='r', linestyle='--', label="{} meV".format(1e3*e_3))
plt.title("A={0}, B={1}, C={2}, D={3}, M={4}, a={5} Lx={6}, Ly={7}"
          .format(params_1['A'], params_1['B'], params_1['C'], params_1['D'],
                 params_1['M'], a, Lx, Ly))

minorLocator = AutoMinorLocator()
ax.yaxis.set_minor_locator(minorLocator)
plt.grid(True, alpha=0.5)

plt.xlabel(r"$k$", fontsize=16)
plt.ylabel(r"$E (\textnormal{eV})$", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(loc='upper left', fontsize=14);
plt.savefig("bstructure_params_Lx={0}_Ly={1}_M={2}.png".format(Lx, Ly, params_1['M']), dpi=196)

In [ ]:
fig, ax = plt.subplots(figsize=(8,6), dpi=196)
kwant.plotter.bands(syst.leads[0], params=params_2,
                    momenta=np.linspace(-0.3,0.3, 150),
                    ax=ax);

ax.axhline(e_1, color='g', linestyle='--', label="{} meV".format(1e3*e_1))
ax.axhline(e_2, color='purple', linestyle='--', label="{} meV".format(1e3*e_2))
ax.axhline(e_3, color='r', linestyle='--', label="{} meV".format(1e3*e_3))

plt.title("A={0}, B={1}, C={2}, D={3}, M={4}, a={5} Lx={6}, Ly={7}"
          .format(params_2['A'], params_2['B'], params_2['C'], params_2['D'],
                 params_2['M'], a, Lx, Ly))

minorLocator = AutoMinorLocator()
ax.yaxis.set_minor_locator(minorLocator)
plt.xlabel(r"$k$", fontsize=16)
plt.ylabel(r"$E (\textnormal{eV})$", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.grid(True, alpha=0.5)


plt.legend(loc='upper left', fontsize=14);
plt.savefig("bstructure_params_Lx={0}_Ly={1}_M={2}.png".format(Lx, Ly, params_2['M']), dpi=196)

In [ ]:
def get_transmissions(syst, energy=0, params=params_1, Wmin=0.1, Wmax=100, npoints=30, nsamples=1, rs=None):
  if rs != None:
    np.random.seed(rs)
    
  Ws = np.logspace(np.log10(Wmin), np.log10(Wmax), npoints)
  
  G1=[]
  G2=[]
  G3=[]
  for W in Ws:
    print_t("W={}".format(W))
    def disorder(x,y):
      return np.random.uniform(-W/2, W/2)
    
    params['V'] = disorder
    #params_2['V'] = disorder
    #params_3 = dict(params_1)
    
    t1 = []
    t2 = []
    t3 = []
    for k in range(nsamples):
      t1.append(kwant.smatrix(syst, energy=e_1, params=params).transmission(1,0))
      t2.append(kwant.smatrix(syst, energy=e_2, params=params).transmission(1,0))
      t3.append(kwant.smatrix(syst, energy=e_3, params=params).transmission(1,0))
      
    G1.append([np.mean(t1), np.std(t1)])
    G2.append([np.mean(t2), np.std(t2)])
    G3.append([np.mean(t3), np.std(t3)])
    
  return Ws, G1, G2, G3

In [ ]:
n_samples = 1
Wmax=500
npoints=60
nsamples=2

In [ ]:
Ws, G1, G2, G3 = get_transmissions(syst, params=dict(params_1), Wmin=0.1, Wmax=Wmax, npoints=npoints, nsamples=n_samples)

In [ ]:
plt.figure(figsize=(8,6), dpi=196)
plt.errorbar(Ws, [g[0] for g in G1], [g[1] for g in G1], linestyle = '-', marker='^', capsize=2,
            label=r"$E_F={}$ meV".format(1e3*e_1))
plt.errorbar(Ws, [g[0] for g in G2], [g[1] for g in G2], linestyle = '-', marker='o', capsize=2,
            label=r"$E_F={}$ meV".format(1e3*e_2))
plt.errorbar(Ws, [g[0] for g in G3], [g[1] for g in G3], linestyle = '-', marker='s', capsize=2,
            label=r"$E_F={}$ meV".format(1e3*e_3))
plt.xscale('log')
plt.xlabel(r"$W$ $(\textnormal{meV})$", fontsize=16)
plt.ylabel(r"$G$ $(e^2/h)$", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.legend(loc='upper right');
plt.title("Lx={0}, Ly={1}, M={2}".format(Lx, Ly, params_1['M']))
plt.savefig("tai_result_Lx={0}_Ly={1}_M={2}_samples={3}.png".format(Lx, Ly, params_1['M'], n_samples), dpi=196)

In [ ]:
Ws, G4, G5, G6 = get_transmissions(syst, params=dict(params_2), Wmin=0.1, Wmax=400, npoints=30, nsamples=n_samples)

In [ ]:
plt.figure(figsize=(8,6), dpi=196)
plt.errorbar(Ws, [g[0] for g in G1], [g[1] for g in G4], linestyle = '-', marker='^', capsize=2,
            label=r"$E_F={}$ meV".format(1e3*e_1))
plt.errorbar(Ws, [g[0] for g in G2], [g[1] for g in G5], linestyle = '-', marker='o', capsize=2,
            label=r"$E_F={}$ meV".format(1e3*e_2))
plt.errorbar(Ws, [g[0] for g in G3], [g[1] for g in G6], linestyle = '-', marker='s', capsize=2,
            label=r"$E_F={}$ meV".format(1e3*e_3))
plt.xscale('log')
plt.legend(loc='upper right');
plt.xlabel(r"$W$ $(\textnormal{meV})$", fontsize=16)
plt.ylabel(r"$G$ $(e^2/h)$", fontsize=16)
plt.tick_params(axis='both', which='major', labelsize=14)
plt.title("Lx={0}, Ly={1}, M={2}".format(Lx, Ly, params_2['M']))
plt.savefig("tai_result_Lx={0}_Ly={1}_M={2}_samples={3}.png".format(Lx, Ly, params_2['M'], n_samples), dpi=196);

In [ ]:
p=dict(A=2, B=2, C=7)
pp = dict(p)
pp['C'] = 8
p

In [ ]:
np.sum([2**k for k in range(7)])

In [ ]:
[k for k in range(7)]

In [ ]:
M=990
N=100
p=2*M/(N*(N-1))
p

In [ ]:
"params={}".format(''.join(repr(dict(A=2, B=2, C=7)).split(" ")))

In [17]:
import multiprocessing as mp
from functools import partial

def pw(x, n):
  a=0
  for i in range(1000):
    a=x**n
  return a

pw_2 = partial(pw, n=2)

arr = np.linspace(0, 100, 10000)

n_cpus = 2
pool = mp.Pool(processes=n_cpus)

In [18]:
%%timeit
results = pool.map(pw_2, arr)

2.02 s ± 307 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%%timeit
[pw_2(x) for x in np.linspace(0, 100, 10000)]

3.25 s ± 244 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
a = [[[2,3], [3,4], [9,9]],  [[2,3], [3,4], [9,9]], [[2,3], [3,4], [9,9]]]
a = np.array(a)

In [22]:
np.savetxt("alma.txt", a)

ValueError: Expected 1D or 2D array, got 3D array instead